In [1]:
import pandas as pd
from sqlalchemy import create_engine
import requests

In [2]:
from config import password

SyntaxError: invalid syntax (config.py, line 2)

In [3]:
# This is the website where we will pull box office numbers for hollywood movies
url = 'https://www.the-numbers.com/box-office-records/domestic/all-movies/cumulative/all-time-inflation-adjusted'

# These headers will allow us to avoid a 403 error by mimicing a web browser
header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}

In [4]:
# format the request to mimic a web browser
r = requests.get(url, headers=header)

# import tables using pandas
tables = pd.read_html(r.text)

# save the first table as a dataframe
movie_numbers_df = tables[0]

movie_numbers_df.head()

,Rank,Released,Movie,Total Box Office
0,1,1977,Star Wars Ep. IV: A New Hope,"$1,497,057,186"
1,2,1997,Titanic,"$1,240,601,558"
2,3,1982,E.T. the Extra-Terrestrial,"$1,224,582,698"
3,4,2015,Star Wars Ep. VII: The Force Awakens,"$1,006,872,847"
4,5,2009,Avatar,"$899,243,135"


In [5]:
# set the initial count value. This will be appended to the URL for each loop to get the 
# next page of movies.
count = 101

# iterate through 40 the movie pages, updating the counter for the next set of 100 movies
while count < 4001:
    url_string = url + f'/{count}'
#     print(url_string)
    req = requests.get(url_string, headers=header)
    tables = pd.read_html(req.text)
    movie_numbers_df = movie_numbers_df.append(tables[0])
    count = count + 100

In [6]:
movie_numbers_df.shape

(4000, 4)

In [7]:
# set the movie ranking to the index
movie_numbers_df.set_index('Rank', inplace=True)
movie_numbers_df.tail()

,Released,Movie,Total Box Office
Rank,,,
3996,1988,Hellbound: Hellraiser II,"$26,017,541"
3997,2018,Searching,"$26,015,149"
3998,1997,Rosewood,"$26,009,132"
3999,2004,Eurotrip,"$25,992,433"
4000,2017,The House,"$25,983,815"


Pull using previous API pull from OMDB

Create database connection

In [ ]:
connection_string = "postgres:password@localhost:5432/customer_db"
engine = create_engine(f'postgresql://{connection_string}')